In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Multithreaded Architectures

Multithreaded processors are everywhere!!! Take a look of our processor again!

In [3]:
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          24
On-line CPU(s) list:             0-23
Thread(s) per core:              1
Core(s) per socket:              16
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           183
Model name:                      13th Gen Intel(R) Core(TM) i7-13700
Stepping:                        1
CPU MHz:                         2100.000
CPU max MHz:                     5200.0000
CPU min MHz:                     800.0000
BogoMIPS:                        4224.00
Virtualization:                  VT-x
L1d cache:                       384 KiB
L1i cache:                       256 KiB
L2 cache:                        16 MiB
NUMA node0 CPU(s):               

## Share memory

In [4]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#else","#endif"])])

In [5]:
! make clean; make testloop; make testloop_O3

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
gcc -O0 -g testloop.c -pthread -lpthread -o testloop 
gcc  -O3 -g testloop.c -pthread -lpthread -o testloop_O3
testloop.c: In function ‘modifyloop’:
testloop.c:23:3: warning: ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
   23 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


In [6]:
! echo 100 | ./testloop 

Please input a number:
Finished. User input is: 100


In [7]:
! echo 10 | ./testloop_O3 

Please input a number:
^C


In [8]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#ifdef","#else"])])

In [10]:
! make clean; make testloop_volatile; make testloop_O3_volatile

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
gcc -O0 -g testloop.c -pthread -lpthread -o testloop_volatile 
gcc  -O3 -DVOLATILE testloop.c -pthread -lpthread -o testloop_O3_volatile
testloop.c: In function ‘modifyloop’:
testloop.c:13:3: warning: ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
   13 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


In [11]:
! echo 10 | ./testloop_volatile

Please input a number:
Finished. User input is: 10


In [12]:
! echo 10 | ./testloop_O3_volatile

Please input a number:
Finished. User input is: 10


## Coherency 

Coherency is the protocol maintaining what values should be seen in the system.

Can you read the following code and guess what's the output?

In [13]:
render_code("coherence.c")

// coherence.c:1-34 (34 lines)
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

volatile int loop;

void* modifyloop(void *x)
{
  sleep(1);

  while(loop < 100)
  {
    loop++;
  }
  return NULL;
}

int main()
{
  pthread_t thread;
  loop = 1;
  
  pthread_create(&thread, NULL, modifyloop, NULL);
  while(loop<100)
  {
//    if(loop < 1000)
    fprintf(stderr,"%d ", loop);
  }
  pthread_join(thread, NULL);
  
  fprintf(stderr,"%4d finished\n", loop);
  return 0;
}

Let's run it several times and see what do we get!

In [14]:
! make coherence; ./coherence >& coherence.out; tail -c 1000 coherence.out

gcc -O3 coherence.c -pthread -lpthread -o coherence 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [15]:
!./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 12  100 finished


### Even with locks...

In [ ]:
render_code("coherence_lock.c")

In [ ]:
! make coherence_lock; ./coherence_lock >& coherence_lock.out; tail -c 1000 coherence_lock.out

In [ ]:
!./coherence_lock >& coherence_lock.out; tail -c 1000 coherence_lock.out; ./coherence_lock >& coherence_lock.out; tail -c 1000 coherence_lock.out; ./coherence_lock >& coherence_lock.out; tail -c 1000 coherence_lock.out; 

### Share memory example

In [3]:
compare([do_render_code("vadd_mt.c",show=["// interleaving","#endif"]),do_render_code("vadd_mt.c",show=["CHUNK","#else"])])

In [21]:
! make clean; make vadd_mt_chunk; make vadd_mt

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration -DHAVE_LINUX_PERF_EVENT_H -pthread -DSSE -DCHUNK vadd_mt.c perfstats.c -o vadd_mt_chunk 
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration -DHAVE_LINUX_PERF_EVENT_H -pthread -DSSE vadd_mt.c perfstats.c -o vadd_mt


In [24]:
!  ssh htseng@ninetales "lscpu"
! echo "size,threads,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "16777216,4," >> stats.csv
! echo "interleaving:"; ssh htseng@ninetales "cd ~/courses/CSE142/demo/multiprocessor; perf stat -e cycles,instructions,cache-misses ./vadd_mt"
! echo -n "16777216,4," >> stats.csv
! echo "chunking:"; 
! ssh htseng@ninetales "cd ~/courses/CSE142/demo/multiprocessor; perf stat -e cycles,instructions,cache-misses  ./vadd_mt_chunk"

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   48 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              2
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       AuthenticAMD
CPU family:                      25
Model:                           33
Model name:                      AMD Ryzen 7 5800X3D 8-Core Processor
Stepping:                        2
Frequency boost:                 enabled
CPU MHz:                         2200.000
CPU max MHz:                     3400.0000
CPU min MHz:                     2200.0000
BogoMIPS:                        6786.86
Virtualization:                  AMD-V
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                     

## Consistency

In [16]:
render_code("fence.c", show=["#ifndef MFENCE","endif"])

// fence.c:13-28 (16 lines)
#ifndef MFENCE
void* modifya(void *z)
{
  while(!go);
  a=1;
  x=b;
  return NULL;
}
void* modifyb(void *z)
{
  while(!go);
  b=1;
  y=a;
  return NULL;
}
#endif

In [17]:
! make clean; make fence

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration fence.c -o fence -lpthread


In [18]:
!rm fence.txt;  touch fence.txt
!lscpu | grep "Model name:"
!cd ~/courses/CSE142/demo/multiprocessor; for i in {1..5000}; do ./fence 2>> fence.txt; done; for i in {1..5000}; do ./fence 2>> fence.txt; done;

Model name:                      13th Gen Intel(R) Core(TM) i7-13700


In [19]:
! grep "(0, 0)" fence.txt |wc
! grep "(0, 1)" fence.txt |wc
! grep "(1, 1)" fence.txt |wc
! grep "(1, 0)" fence.txt |wc

      1       4      19
   9944   39776  188936
      0       0       0
     55     220    1045


In [ ]:
! rm fence_amd.txt; touch fence_amd.txt
! ssh htseng@pikachu "lscpu | grep 'Model name:'; cd ~/courses/CSE142/demo/multiprocessor; for i in {1..10000}; do ./fence 2>> fence_amd.txt; done;"
! grep "(0, 0)" fence_amd.txt |wc
! grep "(0, 1)" fence_amd.txt |wc
! grep "(1, 1)" fence_amd.txt |wc
! grep "(1, 0)" fence_amd.txt |wc

### "mfence" instructions:

An instruction that forces all updates must finish before making progress after this instruction.

In [20]:
render_code("fence.c", show=["#ifdef MFENCE", "#endif"])

// fence.c:41-57 (17 lines)
#ifdef MFENCE
#define _update_var(_v, _x) { _v = (_x); asm("mfence"); }
void* modifya(void *z)
{
//  a=1;
  _update_var(a,1);
  x=b;
  return NULL;
}
void* modifyb(void *z)
{
//  b=1;
  _update_var(b,1);
  y=a;
  return NULL;
}
#endif

In [21]:
!make mfence
!rm mfence.txt; touch mfence.txt
!for i in {1..10000}; do ./fence 2>> mfence.txt; done;
#!ssh htseng@gengar "cd ~/courses/CS203/demo/multiprocessor; for i in {1..3000}; do ./mfence 2>> mfence.txt; done;"

cc -DMFENCE -O3 -Wno-format-zero-length -Wno-implicit-function-declaration fence.c -o mfence -lpthread


In [22]:
! grep "(0, 0)" mfence.txt |wc
! grep "(0, 1)" mfence.txt |wc
! grep "(1, 1)" mfence.txt |wc
! grep "(1, 0)" mfence.txt |wc

      1       4      19
   9924   39696  188556
      0       0       0
     75     300    1425


In [24]:
! rm mfence_amd.txt; touch mfence_amd.txt
! ssh htseng@ninetales "cd ~/courses/CSE142/demo/multiprocessor; for i in {1..10000}; do ./mfence 2>> mfence_amd.txt; done;"
! grep "(0, 0)" mfence_amd.txt |wc
! grep "(0, 1)" mfence_amd.txt |wc
! grep "(1, 1)" mfence_amd.txt |wc
! grep "(1, 0)" mfence_amd.txt |wc

      0       0       0
   9996   39984  189924
      0       0       0
      4      16      76
